<a href="https://colab.research.google.com/github/jordanchenml/nbaStat/blob/master/nba_players_pca_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install nba_api
# !pip install chart_studio
# !pip install plotly --upgrade

In [0]:
from nba_api.stats.endpoints import commonallplayers
import pandas as pd
import numpy as np
from IPython.display import display
from numpy.testing import assert_almost_equal


# 將事先預處理過的英雄數據讀取為 pandas 的 DataFrame
# 你可以從同樣的 url 獲得本文 demo 的數據
df_all_player = pd.read_csv("https://bit.ly/2Or4vnl", 
                            sep="\t")
df_active_player = pd.read_csv("https://bit.ly/3b5itoN", 
                               sep="\t")
df_retire_player = pd.read_csv("https://bit.ly/3b5ZxpH", 
                               sep="\t", 
                               index_col="PLAYER_ID")
df_playerID_name_pairs = pd.read_csv("https://bit.ly/2RW7iXI", 
                                     sep="\t", 
                                     index_col="PLAYER_ID")


print("df_all_player.shape:", df_all_player.shape)
print("df_active_player.shape:", df_active_player.shape)
print("df_retire_player.shape:", df_retire_player.shape)
print("df_playerID_name_pairs.shape:", df_playerID_name_pairs.shape)

df_all_player.shape: (4478, 23)
df_active_player.shape: (492, 23)
df_retire_player.shape: (3986, 22)
df_playerID_name_pairs.shape: (4503, 1)


In [0]:
df_active_player = df_active_player.append(df_all_player[df_all_player['PLAYER_ID'] == 977], ignore_index=True)
# df_active_player = df_active_player.append(df_all_player[df_all_player['PLAYER_ID'] == 893], ignore_index=True)

In [0]:
display(df_active_player)

,PLAYER_ID,POSITION,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203500,Center,512,448.0,13731.0,2074,3521,0.589037,0.0,8.0,0.000000,842,1515,0.555775,1766.0,2095.0,3861.0,586,448.0,514.0,728.0,1347,4990
1,1628389,Center-Forward,201,97.0,5015.0,761,1355,0.561623,4.0,33.0,0.121212,474,665,0.712781,412.0,1087.0,1499.0,529,164.0,163.0,322.0,461,2000
2,200746,Center-Forward,996,955.0,34281.0,7864,15990,0.491807,174.0,553.0,0.314647,3560,4388,0.811303,2669.0,5640.0,8309.0,1965,724.0,1101.0,1559.0,2457,19462
3,1629638,Guard,39,0.0,494.0,77,225,0.342222,40.0,117.0,0.341880,17,28,0.607142,7.0,71.0,78.0,73,11.0,7.0,38.0,43,211
4,1628960,Guard,68,2.0,914.0,146,354,0.412429,65.0,190.0,0.342105,75,95,0.789473,8.0,81.0,89.0,68,12.0,7.0,56.0,83,432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,1629027,Guard,128,128.0,4145.0,956,2225,0.429662,318.0,920.0,0.345652,698,834,0.836930,93.0,423.0,516.0,1078,126.0,20.0,536.0,218,2928
489,203469,Forward-Center,409,242.0,9383.0,1315,2565,0.512670,27.0,105.0,0.257142,849,1158,0.733160,797.0,1625.0,2422.0,566,277.0,294.0,439.0,1087,3506
490,1627790,Center,107,27.0,1455.0,260,447,0.581655,0.0,0.0,0.000000,127,176,0.721590,146.0,289.0,435.0,61,21.0,39.0,77.0,163,647
491,1627826,Center,191,97.0,2971.0,564,1020,0.552941,0.0,6.0,0.000000,255,339,0.752212,334.0,673.0,1007.0,168,46.0,149.0,169.0,366,1383


In [0]:
# df_active_player.info(memory_usage="deep")

In [0]:
# 顯示各特徵的平均與標準差
print("各特徵平均與標準差：")
df_active_player_describe = df_active_player.describe().loc[['mean', 'std']]
# display(df_active_player_describe.style.format("{:.2f}"))

各特徵平均與標準差：


In [0]:
# 將'PLAYER_ID'和'POSITION'以外的 21 個特徵全取出
X = df_active_player.iloc[:, 2:]
df_active_player_positions = df_active_player.iloc[:, 0:2]
# print(df_active_player_positions)

## Standardizing

In [0]:
df_X_std = (X - X.mean(axis=0)) / X.std(axis=0, ddof=0)
df_X_std.describe().loc[['mean', 'std']].style.format("{:.2f}")

,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
mean,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [0]:
X_std = df_X_std.to_numpy()

## 1 - Eigendecomposition - Computing Eigenvectors and Eigenvalues

### Covariance Matrix
$\sum = \frac{1}{n - 1}\lgroup(\mathrm{X}-\bar x)^{T} (\mathrm{X}-\bar x)\rgroup$

In [0]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
# Compute covariance matrix by numpy built in func.
# cov_mat = np.cov(X_std.T)

# print('Covariance matrix \n%s' %cov_mat)

eig_vals, eig_vecs = np.linalg.eig(cov_mat)

# print('Eigenvectors \n%s' %eig_vecs)
# print('\nEigenvalues \n%s' %eig_vals)

### Correlation Matrix

In [0]:
# cor_mat1 = np.corrcoef(X_std.T)

# eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

# print('Eigenvectors \n%s' %eig_vecs)
# print('\nEigenvalues \n%s' %eig_vals)

In [0]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

# Visually confirm that the list is correctly sorted by decreasing eigenvalues
print('Eigenvalues in descending order:')
for i in eig_pairs:
    print(i[0])

Eigenvalues in descending order:
14.877405303183133
2.240317084784821
1.4314964411517093
0.680537202511567
0.4895562241112943
0.43503070308287795
0.32202158241782647
0.1443603240835781
0.10385638945060005
0.09743641375369533
0.07339650853859404
0.04697095581416318
0.03976846855949815
0.030210095894429072
0.0177052749035181
0.0049796985744326085
0.0040613804709336565
0.002780491633269203
0.0007923839093383448
1.9230404481655498e-16
4.579210555233597e-17


In [0]:
import plotly.offline as py

tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = dict(
    type='bar',
    x=['PC %s' %i for i in range(1,22)],
    y=var_exp,
    name='Individual'
)

trace2 = dict(
    type='scatter',
    x=['PC %s' %i for i in range(1,22)], 
    y=cum_var_exp,
    name='Cumulative'
)

data = [trace1, trace2]

layout=dict(
    title='Explained variance by different principal components',
    yaxis=dict(
        title='Explained variance in percent'
    ),
    annotations=list([
        dict(
            x=1.16,
            y=1.05,
            xref='paper',
            yref='paper',
            text='Explained Variance',
            showarrow=False,
        )
    ])
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='selecting-principal-components')

The plot above clearly shows that most of the variance (70.46% of the variance to be precise) can be explained by the first principal component alone. The first six principal components contain 95.66% of the information.

### Projection Matrix
The construction of the projection matrix that will be used to transform the data onto the new feature subspace.

In [0]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(21,1), 
                      eig_pairs[1][1].reshape(21,1), 
                      eig_pairs[2][1].reshape(21,1)))

# print('Matrix W:\n', matrix_w)
Y = X_std.dot(matrix_w)

Visualizing PCA results by 3d scatter chart.

In [0]:
import plotly.graph_objects as go

data = []

colors = {'Center': '#0D76BF', 
          'Center-Forward': '#804040', 
          'Guard': '#EF553B', 
          'Forward': '#48c0a3', 
          'Forward-Guard': '#f0c239', 
          'Guard-Forward': '#161823', 
          'Forward-Center': '#827100'}

for position, col in zip(('Center', 
                          'Center-Forward', 
                          'Guard', 
                          'Forward', 
                          'Forward-Guard', 
                          'Guard-Forward', 
                          'Forward-Center'), colors.values()):
    index = df_active_player_positions.index[df_active_player_positions['POSITION'] == position]
    playerID = df_active_player_positions[df_active_player_positions['POSITION'] == position]['PLAYER_ID']
    trace = go.Scatter3d(
        x = Y[index, 0],
        y = Y[index, 1],
        z = Y[index, 2],
        text = list(df_playerID_name_pairs.loc[playerID]['NAME']),
        mode = 'markers',
        name = position,
        marker = dict(
            color = col,
            size = 12,
            line = dict(
                color = 'rgba(217, 217, 217, 0.14)',
                width = 0.5),
                opacity = 0.8
        )
    )
    data.append(trace)

layout = dict(
    showlegend = True,
    scene = dict(
        xaxis = dict(title = 'PC1'),
        yaxis = dict(title = 'PC2'),
        zaxis = dict(title = 'PC3')
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='projection.html')

## Shortcut - PCA in scikit-learn

In [0]:
print(X_std.shape)

(493, 21)


In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 使用 scikit-learn 內建的 API 正規化
scaler = StandardScaler()
X_std_sk = scaler.fit_transform(X)

assert_almost_equal(X_std, X_std_sk)

# 我們只要最大的兩個主成分。scikit-learn 會自動幫我們
# 依照 eigenvalue 的大小排序共變異數矩陣的 eigenvectors
n_components = 3
random_state = 9527

pca = PCA(n_components=n_components, 
          random_state=random_state)

# 對正規化後的特徵 X_std_sk 做PCA
Y_sk = pca.fit_transform(X_std_sk)  # (n_samples, n_components)

# 將投影到第一主成分的 repr. 顯示在 x 軸，第二主成分在 y 軸
# plt.scatter(L[:, 0], L[:, 1], L[:, 2])

data_sk = []

for position, col in zip(('Center', 
                          'Center-Forward', 
                          'Guard', 
                          'Forward', 
                          'Forward-Guard', 
                          'Guard-Forward', 
                          'Forward-Center'), colors.values()):
    index = df_active_player_positions.index[df_active_player_positions['POSITION'] == position]
    playerID = df_active_player_positions[df_active_player_positions['POSITION'] == position]['PLAYER_ID']
    trace_sk = go.Scatter3d(
        x = Y_sk[index, 0],
        y = Y_sk[index, 1],
        z = Y_sk[index, 2],
        text = list(df_playerID_name_pairs.loc[playerID]['NAME']),
        mode = 'markers',
        name = position,
        marker = dict(
            color = col,
            size = 12,
            line = dict(
                color = 'rgba(217, 217, 217, 0.14)',
                width = 0.5),
                opacity = 0.8
        )
    )
    data_sk.append(trace_sk)

layout = dict(
    showlegend = True,
    scene = dict(
        xaxis = dict(title = 'PC1'),
        yaxis = dict(title = 'PC2'),
        zaxis = dict(title = 'PC3')
    )
)

fig_sk = go.Figure(data=data_sk, layout=layout)
py.iplot(fig_sk, filename='projection_sk.html')

In [0]:
X_cp = X_std_sk.copy()
print(type(X_cp))

<class 'numpy.ndarray'>


In [0]:
from sklearn.manifold import TSNE
import time

pca = PCA(n_components=10)
pca_result = pca.fit_transform(X_std_sk)
print('Cumulative explained variation for n principal components: {}'.format(np.sum(pca.explained_variance_ratio_)))
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
Y_tsne = tsne.fit_transform(pca_result)

Cumulative explained variation for n principal components: 0.9895134446940302
[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 493 samples in 0.001s...
[t-SNE] Computed neighbors for 493 samples in 0.022s...
[t-SNE] Computed conditional probabilities for sample 493 / 493
[t-SNE] Mean sigma: 0.683836
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.821404
[t-SNE] KL divergence after 300 iterations: 2.250958


In [0]:
data_tsne = []

for position, col in zip(('Center', 
                          'Center-Forward', 
                          'Guard', 
                          'Forward', 
                          'Forward-Guard', 
                          'Guard-Forward', 
                          'Forward-Center'), colors.values()):
    index = df_active_player_positions.index[df_active_player_positions['POSITION'] == position]
    playerID = df_active_player_positions[df_active_player_positions['POSITION'] == position]['PLAYER_ID']
    trace_tsne = go.Scatter3d(
        x = Y_tsne[index, 0],
        y = Y_tsne[index, 1],
        z = Y_tsne[index, 2],
        text = list(df_playerID_name_pairs.loc[playerID]['NAME']),
        mode = 'markers',
        name = position,
        marker = dict(
            color = col,
            size = 12,
            line = dict(
                color = 'rgba(217, 217, 217, 0.14)',
                width = 0.5),
                opacity = 0.8
        )
    )
    data_tsne.append(trace_tsne)

layout = dict(
    showlegend = True,
    scene = dict(
        xaxis = dict(title = 'PC1'),
        yaxis = dict(title = 'PC2'),
        zaxis = dict(title = 'PC3')
    )
)

fig_tsne = go.Figure(data=data_tsne, layout=layout)
py.iplot(fig_tsne, filename='projection_tsne.html')

In [0]:
import chart_studio
import chart_studio.plotly as py_studio

username = ''
api_key = ''

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py_studio.plot(fig, filename = 'projection', auto_open=True)


'https://plot.ly/~jordanchenml/3/'